# Ejercicio Práctico : Soft Computing

Autores: Danel Arias y Rubén Pérez
Fecha: 30 de enero 2024

--------------------------


In [44]:
# imports
import numpy as np

## Enunciado
Supongamos que cuatro expertos, $𝐸=\{𝑒_1, 𝑒_2, 𝑒_3, 𝑒_4\}$, deben elegir la mejor alternativa de entre un conjunto de cuatro posibles, $𝑋=\{𝑥_1, 𝑥_2, 𝑥_3, 𝑥_4\}$. Para ello, expresan sus opiniones mediante las siguientes relaciones de 
preferencia difusas:

$p^1 = \begin{pmatrix}- & 0.5 & 0.7 & 0.8 \\
0.5 &- & 0.6 & 0.1\\
0.2 & 0.3 &- & 0.5\\
0.2 & 0.8 & 0.5 &-\\
\end{pmatrix}$

$p^2 = \begin{pmatrix}- & 0.9 & 0.2 & 0.2 \\
0.1 &- & 0.3 & 0.9\\
0.7 & 0.7 &- & 0.6\\
0.7 & 0.1 & 0.4 &-\\
\end{pmatrix}$

$p^3 = \begin{pmatrix}- & 0.7 & 0.8 & 0.6 \\
0.3 &- & 0.7 & 0.2\\
0.1 & 0.3 &- & 0.3\\
0.3 & 0.7 & 0.6 &-\\
\end{pmatrix}$

$p^4 = \begin{pmatrix}- & 0.2 & 0.8 & 0.8 \\
0.7 &- & 0.6 & 0.3\\
0.1 & 0.3 &- & 0.5\\
0.1 & 0.7 & 0.5 &-\\
\end{pmatrix}$

In [45]:
# Representación de las matrices en numpy
p1 = np.array([[np.nan, 0.5, 0.7, 0.8],
               [0.5, np.nan, 0.6, 0.1],
               [0.2, 0.3, np.nan, 0.5],
               [0.2, 0.8, 0.5, np.nan]])
p2 = np.array([[np.nan, 0.9, 0.2, 0.2],
               [0.1, np.nan, 0.3, 0.9],
               [0.7, 0.7, np.nan, 0.6],
               [0.7, 0.1, 0.4, np.nan]])
p3 = np.array([[np.nan, 0.7, 0.8, 0.6],
               [0.3, np.nan, 0.7, 0.2],
               [0.1, 0.3, np.nan, 0.3],
               [0.3, 0.7, 0.6, np.nan]])
p4 = np.array([[np.nan, 0.2, 0.8, 0.8],
               [0.7, np.nan, 0.6, 0.3],
               [0.1, 0.3, np.nan, 0.5],
               [0.1, 0.7, 0.5, np.nan]])
pref = np.array([p1, p2, p3, p4])
n_x = p1.shape[1]

## Preguntas a responder

> Nota:  Use  la  relación  de  preferencia  colectiva  obtenida  en  el  punto  2  para   calcular  los  grados  de  dominancia  y  de  no  dominancia  solicitados  en  los puntos 3 y 4.

1. ¿Cuál es el nivel de consenso alcanzado entre los cuatro expertos?

In [46]:
# Cálculo de la matriz de consenso
sms =[]
for i in range(len(pref)):
    for j in range(i+1, len(pref)):
        sm = 1 - np.abs(pref[i] - pref[j])
        print(f'P{i+1} - P{j+1}:\n{sm}')
        sms.append(sm)

consensus = np.mean(sms, axis=0)


P1 - P2:
[[nan 0.6 0.5 0.4]
 [0.6 nan 0.7 0.2]
 [0.5 0.6 nan 0.9]
 [0.5 0.3 0.9 nan]]
P1 - P3:
[[nan 0.8 0.9 0.8]
 [0.8 nan 0.9 0.9]
 [0.9 1.  nan 0.8]
 [0.9 0.9 0.9 nan]]
P1 - P4:
[[nan 0.7 0.9 1. ]
 [0.8 nan 1.  0.8]
 [0.9 1.  nan 1. ]
 [0.9 0.9 1.  nan]]
P2 - P3:
[[nan 0.8 0.4 0.6]
 [0.8 nan 0.6 0.3]
 [0.4 0.6 nan 0.7]
 [0.6 0.4 0.8 nan]]
P2 - P4:
[[nan 0.3 0.4 0.4]
 [0.4 nan 0.7 0.4]
 [0.4 0.6 nan 0.9]
 [0.4 0.4 0.9 nan]]
P3 - P4:
[[nan 0.5 1.  0.8]
 [0.6 nan 0.9 0.9]
 [1.  1.  nan 0.8]
 [0.8 1.  0.9 nan]]


In [47]:
print(f'Matriz de consenso:\n{consensus}')

Matriz de consenso:
[[       nan 0.61666667 0.68333333 0.66666667]
 [0.66666667        nan 0.8        0.58333333]
 [0.68333333 0.8               nan 0.85      ]
 [0.68333333 0.65       0.9               nan]]


In [48]:
global_consensus = np.mean(consensus[~np.isnan(consensus)])
print(f'Consenso global:\n{global_consensus}')

Consenso global:
0.7152777777777778


2. Usando el operador OWA con el cuantificador lingüístico difuso «mayoría», definido por $𝑄(𝑟) = 𝑟^{1/2}$, para obtener sus pesos, ¿cuál es la
relación de preferencia colectiva?

In [49]:
def q(n):
    return np.sqrt(n)
def calc_w(n):
    w = np.ndarray(n)
    for i in range(1, n+1):
        w[i-1] = q(i / n) - q((i-1) / n)
    return w

n_exp = len(pref) # número de expertos
w4 = calc_w(n_exp)

In [50]:
def psi_q(array_p, w):
    # Ordenar de mayor a menor
    array_p = np.sort(array_p)[::-1]
    # Aplicar los pesos
    return np.sum(array_p * w)


In [51]:
# Ordenamos las matrices de preferencia de mayor a menor y aplicamos las ponderaciones
pref_ag = np.ndarray((n_x, n_x))
for i in range(n_x):
    for j in range(n_x):
        # Ordenamos los elementos i,j de cada una de las matrices de preferencia
        pref_ij = np.array([p[i, j] for p in pref])
        pref_ag[i, j] = psi_q(pref_ij, w4)

print(f'Matriz de preferencia:\n{pref_ag}')

Matriz de preferencia:
[[       nan 0.70122898 0.70372338 0.68783152]
 [0.51462644        nan 0.60980762 0.55731322]
 [0.42071068 0.5               nan 0.52320508]
 [0.45731322 0.66961524 0.53660254        nan]]


3. ¿Cuál es el grado de dominancia guiado por cuantificador asociado a cada alternativa? Indica el ranking de alternativas obtenido. Utilice de nuevo el
operador OWA con el cuantificador lingüístico difuso «mayoría», definido por $𝑄(𝑟) = 𝑟^{1/2}$, para obtener sus pesos.

In [52]:
# Los pesos son los mismos que en el caso anterior
# Calculamos el grado de dominancia
dom = np.ndarray((n_x))
w3 = calc_w(n_exp - 1)
for i in range(n_x):
    fila = pref_ag[i, :]
    # Quitar columna i
    fila = np.delete(fila, i)
    dom[i] = psi_q(fila, w3)
print(f'Matriz de dominancia:\n{dom}')


Matriz de dominancia:
[0.70021064 0.57978771 0.4988476  0.5988476 ]


4. ¿Cuál es el grado de **no** dominancia guiado por cuantificador asociado a cada alternativa? Indica el ranking de alternativas obtenido. Use de nuevo
el operador OWA con el cuantificador lingüístico difuso «mayoría», definido por $𝑄(𝑟) = 𝑟^{1/2}$, para obtener sus pesos

In [56]:
# Los pesos son los mismos que en el caso anterior
# Calculamos el grado de NO dominancia
no_dom = np.zeros((n_x))
# Calculamos haciendo el max(0, diferencia entre los elementos simétricos frente a la diagonal)
for col in range(n_x):
    res = np.ndarray((n_x))
    for i in range(n_x):
        if i != col:
            el = pref_ag[i, col]
            sim = pref_ag[col, i]
            res[i] = 1 - max(0, el - sim)

    # Ordenamos los elementos de res de mayor a menor y aplicamos las ponderaciones
    no_dom[col] = psi_q(res, w4)

print(f'Matriz de NO dominancia:\n{no_dom}')


Matriz de NO dominancia:
[0.98205081 0.95715312 0.94185831 0.9691164 ]
